In [ ]:
# First time running ? In Terminal: pip install sodapy 

from sodapy import Socrata
import pandas as pd

In [2]:
# This will take a few minutes
# Documentation, code snippet, and token information: https://dev.socrata.com/foundry/datacatalog.cookcountyil.gov/apwk-dzx8
# This is based heavily on an NYU presentation from Nicholas Wolf and Vicky Steeves-
# Python for Harvesting Data on the Web - https://nyu-dataservices.github.io/DataHarvesting-Python/

client = Socrata('datacatalog.cookcountyil.gov',
                  'UEcAAUQpAzK6Dl3x7dYTCc5cc',
                  username="kelseymarkey@gmail.com",
                  password="Idsproject1!")

init = client.get("7mck-ehwz", limit=1000000)
disp = client.get("apwk-dzx8", limit=900000)
sent = client.get("tg8v-tm6u", limit=300000)

initiation = pd.DataFrame.from_records(init)
disposition = pd.DataFrame.from_records(disp)
sentencing = pd.DataFrame.from_records(sent)

Ideally we would have a MultiIndexed dataframe comprised of an outer index which is CASE_PARTICIPANT_ID, and an inner index which is CHARGE_ID.

TARGET VARIABLE: 
Our target variable will be a binary indicator variable, “Mental Health Indicator” (MHI), that we will construct using “OR” statements. An MHI of 1 will indicate that the participant had a mental health related disposition or sentence. More specifically, a row will be assigned an MHI of 1 if any of the following features have any of the following values:

From Disposition Data:
CHARGE_DISPOSITION: “FNG Reason Insanity” (532 instances); “Finding Guilty but Mentally Ill” (37 instances); “Plea of Guilty but Mentally Ill” (30 instances); “Verdict Guilty But Mental Ill” (1 instance) 
CHARGE_DISPOSITION_REASON: “Mental Health Graduate” (208 instances). 

Sentencing
SENTENCE_TYPE:  “Inpatient Mental Health Services” (212 instances);
CHARGE_DISPOSITION: “FNG Reason Insanity” (131 instances); “Finding Guilty but Mentally Ill” (39 instances); “Plea of Guilty but Mentally Ill” (30 instances); “Verdict Guilty but Mentally Ill” (1 instances)
CHARGE_DISPOSITION_REASON: “Mental Health Graduate” (208 instances)
COMMITMENT_TYPE: “Mental Health Probation” (2226 instances), “Inpatient Mental Health Services” (181 instances)

In [3]:
sentencing.head()

,act,age_at_incident,aoic,arraignment_date,arrest_date,case_id,case_participant_id,chapter,charge_count,charge_disposition,...,primary_charge,race,received_date,section,sentence_date,sentence_judge,sentence_phase,sentence_type,unit,updated_offense_category
0,-,27,0000001607,9/21/1984 12:00:00 AM,8/15/1984 12:00:00 AM,841463847080,139918426161,38,2,Nolle On Remand,...,False,Black,8/15/1984 12:00:00 AM,9-1(a)(2),6/2/1986 12:00:00 AM,John Mannion,Original Sentencing,Conversion,NaN,Homicide
1,-,27,0000001608,9/21/1984 12:00:00 AM,8/15/1984 12:00:00 AM,841463847080,139918426161,38,4,Nolle On Remand,...,False,Black,8/15/1984 12:00:00 AM,9-1(a)(3),6/2/1986 12:00:00 AM,John Mannion,Original Sentencing,Conversion,NaN,Homicide
2,-,27,0000001608,9/21/1984 12:00:00 AM,8/15/1984 12:00:00 AM,841463847080,139918426161,38,5,Nolle On Remand,...,False,Black,8/15/1984 12:00:00 AM,9-1(a)(3),6/2/1986 12:00:00 AM,John Mannion,Original Sentencing,Conversion,NaN,Homicide
3,NaN,27,0000001846,9/21/1984 12:00:00 AM,8/15/1984 12:00:00 AM,841463847080,139918426161,38-12-11-A(1),13,Plea Of Guilty,...,False,Black,8/15/1984 12:00:00 AM,NaN,10/16/2014 12:00:00 AM,Clayton Jay Crane,Amended/Corrected Sentencing,Prison,NaN,Homicide
4,NaN,27,0000001846,9/21/1984 12:00:00 AM,8/15/1984 12:00:00 AM,841463847080,139918426161,38-12-11-A(1),13,Plea Of Guilty,...,False,Black,8/15/1984 12:00:00 AM,NaN,6/2/1986 12:00:00 AM,John Mannion,Original Sentencing,Conversion,NaN,Homicide


In [4]:
#check if values in cols are unique
sentencing.aoic.is_unique

False

In [5]:
# Check which cells in column NaN
len(sentencing.loc[sentencing.age_at_incident.isnull()]) #remove len() to see specific rows
# sentencing.loc[sentencing.case_id.isnull()] - 0 rows

2946

In [6]:
# look at value counts of specific column 
sentencing.charge_disposition.value_counts()
# also look into groupby: https://www.geeksforgeeks.org/python-pandas-dataframe-groupby/ 

Plea Of Guilty                      202789
Finding Guilty                       20700
Verdict Guilty                        4427
Case Dismissed                         535
Nolle Prosecution                      494
FNG Reason Insanity                    131
Finding Not Not Guilty                  97
Charge Vacated                          51
Finding Guilty But Mentally Ill         39
FNG                                     37
Finding Guilty - Lesser Included        35
Plea of Guilty - Amended Charge         33
Plea of Guilty But Mentally Ill         30
BFW                                     30
Death Suggested-Cause Abated            17
Nolle On Remand                         13
Plea of Guilty - Lesser Included         8
Sexually Dangerous Person                7
Verdict-Not Guilty                       7
Verdict Guilty - Lesser Included         5
WOWI                                     4
SOL                                      2
Finding Guilty - Amended Charge          2
FNPC       

In [ ]:
# checking uniqueness of charge id vs charge version id in initiation data
initiation.charge_disposition.value_counts()

look at variables unique to multiple-charge cases